# Testing Data Modelling Techniques

This notebook is a complete mess. I'll make it clean later on (still testing some stuff).

Techniques to be tested:
- User2Vec
- Network Analysis (Tweets)
- Network Analysis (Users)
- Popular vs unpopular users identification
- Topic Modelling
- Text summarization

In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import eland as ed
import networkx as nx

# uncomment this if database is not already open (and give ES a couple minutes to set up)
#!make database

es = Elasticsearch(['localhost'])

In [2]:
# drop empty coluns
ed_df = ed.read_es('localhost', 'twitter')

df = ed_df[
    ['tweet_id', 'user_id', 'name', 'full_text_processed',
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator']
].to_pandas().fillna(np.nan)

In [3]:
# remove the annoying scientific notation from id columns
pd.set_option('display.float_format', lambda x: '%.0f' % x)

def get_source_id(row):
    """Returns the original Tweet ID from a Quote, Retweet or Reply"""
    if row['is_quote_status']:
        return row['quoted_status_id_str']
    elif row['is_retweet']:
        return row['original_tweet_id_str']
    elif row['is_reply']:
        return row['in_reply_to_status_id_str']
    else:
        return np.nan
    
df['original_tweet_id_str'] = df.apply(get_source_id, axis=1)
# I'm counting replies as original posts... Please correct me if you disagree
df['is_original'] = ~df[['is_quote_status', 'is_retweet']].max(1)

## Network Analysis (Tweets)
There is possibly little to no information to be taken from this section, but I kept it anyway.

In [16]:
#df_network = ed_df[
#    ['tweet_id', 
#    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
#    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
#].to_pandas().fillna(np.nan).rename(columns={'tweet_id': 'target'})

In [17]:
# remove the annoying scientific notation from id columns
# pd.set_option('display.float_format', lambda x: '%.0f' % x)

df_network['source'] = df_network.apply(get_source, axis=1)

In [20]:
df_network['weight'] = 1
df_network[['source', 'target', 'weight']]

,source,target,weight
1263804753286397952,1263801627288521984,1263804753286397952,1
1263820895312732160,1263812617430134785,1263820895312732160,1
1263820892871618560,1263762563566731264,1263820892871618560,1
1263820887897378816,1263788358343499782,1263820887897378816,1
1263820880800595968,1263820878783053056,1263820880800595968,1
...,...,...,...
1263850896217092096,1263848854383849477,1263850896217092096,1
1263850895160348672,1263848854383849477,1263850895160348672,1
1263850832103161856,1263847781094944768,1263850832103161856,1
1263850818689740800,1263848064592183298,1263850818689740800,1


In [32]:
# create a directed graph
G_dir = nx.from_pandas_edgelist(df_network[['source', 'target', 'weight']], create_using=nx.DiGraph())

# create an undirected graph
G = nx.from_pandas_edgelist(df_network[['source', 'target', 'weight']], create_using=nx.Graph())


In [37]:
print(f'Graph basic info: {G_dir.number_of_nodes()} nodes, {G_dir.number_of_edges()} edges.')

Graph basic info: 495233 nodes, 455471 edges.


In [38]:
# compare densities of the two graphs
print(f'Density measurement: \nUndirected: {nx.density(G)}\nDirected: {nx.density(G_dir)}')

Density measurement: 
Undirected: 3.71426127592167e-06
Directed: 1.857130637960835e-06


In [39]:
#out-degree
node_attrs=pd.DataFrame.from_dict(dict(G_dir.out_degree()), 'index').rename(columns={0:'out_degree'})

#in-degree
node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.in_degree()), 'index').rename(columns={0:'in_degree'}))

# weighted out-degree
#node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.out_degree(weight='weight')), 'index').rename(columns={0:'weighted_out_degree'}))

# weighted in-degree
#node_attrs=node_attrs.join(pd.DataFrame.from_dict(dict(G_dir.in_degree(weight='weight')), 'index').rename(columns={0:'weighted_in_degree'}))

# add tweet ids and original info
#node_attrs=node_attrs.join(df.set_index('COMUNE')[["COD_REG","COD_CM", "COD_PRO", "PRO_COM"]])

In [48]:
node_attrs

,out_degree,in_degree
126,1,0
278816194440605696,1,0
401812009810800576,1,0
446602988501491712,1,0
521237811643445184,1,0
...,...,...
nan,1,0
nan,1,0
nan,1,0
nan,1,0


In [ ]:
node_attrs.out_degree.unique()

In [40]:
df_network[['source', 'target', 'weight']]

,source,target,weight
1263804753286397952,1263801627288521984,1263804753286397952,1
1263820895312732160,1263812617430134785,1263820895312732160,1
1263820892871618560,1263762563566731264,1263820892871618560,1
1263820887897378816,1263788358343499782,1263820887897378816,1
1263820880800595968,1263820878783053056,1263820880800595968,1
...,...,...,...
1263850896217092096,1263848854383849477,1263850896217092096,1
1263850895160348672,1263848854383849477,1263850895160348672,1
1263850832103161856,1263847781094944768,1263850832103161856,1
1263850818689740800,1263848064592183298,1263850818689740800,1


## Network Analysis (users)

In [66]:
df_network = ed_df[
    ['tweet_id', 'name',
    'original_tweet_id_str', 'quoted_status_id_str', 'in_reply_to_status_id_str',
    'is_quote_status', 'is_retweet', 'is_reply', 'is_translator', 'is_original']
].to_pandas().fillna(np.nan)

In [61]:
cols = ed_df.columns
cols[cols.str.contains('name')]

#ed_df[[]]

Index(['derived.locations.full_name', 'entities.user_mentions.name',
       'entities.user_mentions.screen_name', 'name', 'screen_name',
       'users_derived_full_name'],
      dtype='object')

## User2Vec
Based on: https://ieeexplore.ieee.org/document/8875952/

How it works:
- Run doc2vec
- Average vector representations for each user


In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.models.word2vec import LineSentence

# Hyperparameters
vector_size = 10
min_count = 2
epochs = 40

In [5]:
df_doc2vec = df[[
    'tweet_id', 'original_tweet_id_str', 
    'user_id', 'name', 'full_text_processed', 
]][df['is_original']].copy()

In [6]:
df_doc2vec.head()

,tweet_id,original_tweet_id_str,user_id,name,full_text_processed
1263804753286397952,1263804753286397952,1263801627288521984,2739781807,Debjani Bhattacharyya,66 km embankment breached yes need quickly rep...
1263820880800595968,1263820880800595968,1263820878783053056,81572564,Malcolm Sen,crisis singular event yet another chapter unfo...
1263820878783053824,1263820878783053824,NaN,81572564,Malcolm Sen,utter devastation kolkata wake state fails res...
1263820864967135232,1263820864967135232,NaN,1214239561598951428,AfricaZilla,prayer thought love affected cyclone amphan sh...
1263820862827958272,1263820862827958272,NaN,2165333665,Savi,image west bengal tragic water monsoon way


In [7]:
def tokenize(doc, tag=None):
    """Tokenizes a single tweet."""
    tokens = simple_preprocess(doc)
    if tag is None:
        return tokens
    else:
        return TaggedDocument(tokens, [tag])

train_corpus = [tokenize(doc, tag) for doc, tag in zip(df_doc2vec['full_text_processed'], range(df_doc2vec.shape[0]))]

In [8]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = Doc2Vec(vector_size=vector_size, min_count=min_count, epochs=epochs)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-08-13 00:15:30,211 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-08-13 00:15:30,245 : INFO : collecting all words and their counts
2020-08-13 00:15:30,266 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-08-13 00:15:30,731 : INFO : PROGRESS: at example #10000, processed 131046 words (296138/s), 12137 word types, 10000 tags
2020-08-13 00:15:30,937 : INFO : PROGRESS: at example #20000, processed 260493 words (632493/s), 18266 word types, 20000 tags
2020-08-13 00:15:31,048 : INFO : PROGRESS: at example #30000, processed 384004 words (1123313/s), 23084 word types, 30000 tags
2020-08-13 00:15:31,182 : INFO : PROGRESS: at example #40000, processed 502154 words (889773/s), 26517 word types, 40000 tags
2020-08-13 00:15:31,292 : INFO : PROGRESS: at example #50000, processed 622138 words (1100278/s), 29657 word types, 50000 tags
2020-08-13 00:15:31,484 : INFO : PROGRESS: at example #60000, processed 760279 words (

2020-08-13 00:16:44,115 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:16:44,115 : INFO : EPOCH - 5 : training on 1349510 raw words (1225136 effective words) took 8.3s, 148376 effective words/s
2020-08-13 00:16:45,136 : INFO : EPOCH 6 - PROGRESS: at 12.43% examples, 152831 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:16:46,175 : INFO : EPOCH 6 - PROGRESS: at 27.08% examples, 159303 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:16:47,228 : INFO : EPOCH 6 - PROGRESS: at 42.61% examples, 163226 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:16:48,260 : INFO : EPOCH 6 - PROGRESS: at 56.34% examples, 164365 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:16:49,270 : INFO : EPOCH 6 - PROGRESS: at 71.92% examples, 169461 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:16:50,348 : INFO : EPOCH 6 - PROGRESS: at 87.47% examples, 170552 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:16:51,137 : INFO : worker thread finished; awaiting finish of 2 more threads
202

2020-08-13 00:17:35,897 : INFO : EPOCH 13 - PROGRESS: at 30.58% examples, 180287 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:17:36,985 : INFO : EPOCH 13 - PROGRESS: at 48.22% examples, 180527 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:17:38,008 : INFO : EPOCH 13 - PROGRESS: at 63.74% examples, 184926 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:17:39,018 : INFO : EPOCH 13 - PROGRESS: at 78.96% examples, 183581 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:17:40,048 : INFO : EPOCH 13 - PROGRESS: at 95.45% examples, 186739 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:17:40,278 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-13 00:17:40,301 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:17:40,334 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:17:40,335 : INFO : EPOCH - 13 : training on 1349510 raw words (1224627 effective words) took 6.5s, 188474 effective words/s
2020-08-13 00:17:41

2020-08-13 00:18:28,097 : INFO : EPOCH 20 - PROGRESS: at 94.70% examples, 155998 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:18:28,363 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-13 00:18:28,367 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:18:28,386 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:18:28,387 : INFO : EPOCH - 20 : training on 1349510 raw words (1225302 effective words) took 7.7s, 159721 effective words/s
2020-08-13 00:18:29,423 : INFO : EPOCH 21 - PROGRESS: at 14.74% examples, 176963 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:18:30,495 : INFO : EPOCH 21 - PROGRESS: at 30.58% examples, 177494 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:18:31,553 : INFO : EPOCH 21 - PROGRESS: at 47.44% examples, 177634 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:18:32,567 : INFO : EPOCH 21 - PROGRESS: at 63.04% examples, 183084 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:18:33

2020-08-13 00:19:19,099 : INFO : EPOCH 27 - PROGRESS: at 71.92% examples, 166895 words/s, in_qsize 5, out_qsize 0
2020-08-13 00:19:20,114 : INFO : EPOCH 27 - PROGRESS: at 86.79% examples, 168663 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:19:21,115 : INFO : EPOCH 27 - PROGRESS: at 98.65% examples, 166749 words/s, in_qsize 2, out_qsize 1
2020-08-13 00:19:21,119 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-13 00:19:21,131 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:19:21,165 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:19:21,165 : INFO : EPOCH - 27 : training on 1349510 raw words (1225328 effective words) took 7.3s, 168082 effective words/s
2020-08-13 00:19:22,215 : INFO : EPOCH 28 - PROGRESS: at 13.89% examples, 165700 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:19:23,281 : INFO : EPOCH 28 - PROGRESS: at 29.70% examples, 172279 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:19:24

2020-08-13 00:20:08,793 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-13 00:20:08,824 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-13 00:20:08,825 : INFO : EPOCH - 34 : training on 1349510 raw words (1224891 effective words) took 6.5s, 188034 effective words/s
2020-08-13 00:20:09,839 : INFO : EPOCH 35 - PROGRESS: at 14.65% examples, 181243 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:20:10,865 : INFO : EPOCH 35 - PROGRESS: at 30.58% examples, 183487 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:20:11,901 : INFO : EPOCH 35 - PROGRESS: at 47.54% examples, 182826 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:20:12,921 : INFO : EPOCH 35 - PROGRESS: at 62.42% examples, 184618 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:20:13,932 : INFO : EPOCH 35 - PROGRESS: at 78.08% examples, 185124 words/s, in_qsize 6, out_qsize 0
2020-08-13 00:20:14,946 : INFO : EPOCH 35 - PROGRESS: at 92.98% examples, 185595 words/s, in_qsize 5, out_qsiz

In [88]:
doc2vec_cols = [f'doc2vec_{i}' for i in range(vector_size)]

doc_vectors = df_doc2vec['full_text_processed'].apply(
    lambda doc: np.expand_dims(
        model.infer_vector(tokenize(doc)), axis=0
    )
)
df_vectors = pd.DataFrame(
    np.concatenate(doc_vectors, 0), columns=doc2vec_cols
).set_index(df_doc2vec.index)

In [48]:
df_user2vec = df[[
    'tweet_id', 'original_tweet_id_str',
    'user_id', 'name', 'full_text_processed', 
]][~df['is_original']].copy()

def get_vectors(tweet_id):
    try:
        return df_doc2vec.loc[
            tweet_id, 
            doc2vec_cols
        ]
    except KeyError:
        pass

_vectors = df_user2vec['original_tweet_id_str'].apply(get_vectors)

In [51]:
_vectors[~_vectors.isnull()]


Series([], Name: original_tweet_id_str, dtype: object)

In [70]:
df[~df['is_retweet']].shape, df[~df['is_retweet']]['full_text_processed'].unique().shape

((109417, 12), (82378,))

In [67]:
df['full_text_processed'].unique().shape, df_doc2vec.shape, df_doc2vec['full_text_processed'].unique().shape

((83811,), (104775, 5), (78658,))

In [74]:
df['original_tweet_id_str'].astype(str).loc['1263804753286397952']

'1.263801627288522e+18'

In [85]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
df.loc[str(int(df['original_tweet_id_str'].iloc[200]))]

KeyError: '1263788358343499782'

In [87]:
df.columns[df.columns.str.contains('id')]

Index(['tweet_id', 'user_id', 'original_tweet_id_str', 'quoted_status_id_str',
       'in_reply_to_status_id_str'],
      dtype='object')